# Análisis de Rendimiento de Algoritmos de Ordenamiento

Este notebook ejecuta un *profiler* (contador de operaciones) para tres algoritmos de ordenamiento: **Bubble Sort**, **Selection Sort** e **Insertion Sort**, y luego grafica los resultados.

El objetivo es comparar su rendimiento en tres escenarios:
1.  **Caso Promedio** (Lista aleatoria)
2.  **Mejor Caso** (Lista ya ordenada)
3.  **Peor Caso** (Lista en orden inverso)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import copy

# Configurar matplotlib para que se muestre en el notebook
%matplotlib inline

## 1. Definición de Algoritmos y Generadores

Primero, definimos las funciones de ordenamiento (con *profiling* para contar operaciones) y las funciones que generan los casos de prueba.

In [2]:
# --- 1. ALGORITMOS CON PROFILER ---

def bubble_sort_profiled(arr):
    # --- 1. Inicializar contadores ---
    comparisons = 0
    swaps = 0
    
    n = len(arr)
    for i in range(n):
        swapped = False
        for j in range(0, n - i - 1):
            
            # --- 2. Contar la comparación ---
            comparisons += 1
            if arr[j] > arr[j + 1]:
                
                # --- 3. Contar el intercambio ---
                swaps += 1
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
                
        if not swapped:
            break
            
    # --- 4. Devolver los resultados ---
    return arr, comparisons, swaps


def selection_sort_profiled(arr):
    # --- 1. Inicializar contadores ---
    comparisons = 0
    swaps = 0
    
    n = len(arr)
    # El bucle externo solo necesita ir hasta n-1
    for i in range(n - 1):
        
        # Asume que el mínimo es el primer elemento de la parte no ordenada
        min_idx = i
        
        # Bucle interno para encontrar el mínimo real
        for j in range(i + 1, n):
            
            # --- 2. Contar la comparación ---
            comparisons += 1
            if arr[j] < arr[min_idx]:
                min_idx = j # Encontraste un nuevo mínimo
        
        # --- 3. Contar el intercambio (¡fuera del bucle j!) ---
        # Solo intercambia si el mínimo no es ya la posición i
        if min_idx != i:
            swaps += 1
            arr[i], arr[min_idx] = arr[min_idx], arr[i]
            
    # --- 4. Devolver los resultados ---
    return arr, comparisons, swaps

def insertion_sort_profiled(arr):
    # --- 1. Inicializar contadores ---
    comparisons = 0
    shifts = 0  # Movimientos de datos (no intercambios)

    for i in range(1, len(arr)):
        key = arr[i]
        j = i - 1

        # Este bucle 'while' es el motor de comparación y desplazamiento
        while j >= 0:
            
            # --- 2. Contar la comparación ---
            comparisons += 1
            if key < arr[j]:
                
                # --- 3. Contar el desplazamiento ---
                shifts += 1
                arr[j + 1] = arr[j]
                j -= 1
            else:
                # La comparación (key < arr[j]) fue Falsa, 
                # así que encontramos el lugar y salimos del 'while'
                break
        
        arr[j + 1] = key
            
    # --- 4. Devolver los resultados ---
    return arr, comparisons, shifts

# --- 2. GENERADORES DE CASOS DE PRUEBA ---

def generate_average_case(n):
    """Genera una lista de n números únicos, en orden aleatorio."""
    arr = list(range(n))
    random.shuffle(arr)
    return arr

def generate_best_case(n):
    """Genera una lista de n números ya ordenados."""
    return list(range(n))

def generate_worst_case(n):
    """Genera una lista de n números en orden inverso."""
    return list(range(n, 0, -1))

## 2. Ejecución del Testbench y Creación del DataFrame

Ahora, ejecutamos el *testbench*. En lugar de guardar en un CSV, almacenamos los resultados en una lista y la convertimos directamente en un DataFrame de Pandas.

*(Nota: Se han reducido los tamaños máximos para que el notebook se ejecute más rápido. Puedes añadir `3200` y `6400` a la lista `sizes` si deseas un análisis más profundo, pero tardará más.)*

In [3]:
# --- 3. CONFIGURACIÓN DEL TESTBENCH ---

# Lista de algoritmos a probar
algorithms_to_test = [
    ("Bubble Sort", bubble_sort_profiled),
    ("Selection Sort", selection_sort_profiled),
    ("Insertion Sort", insertion_sort_profiled)
]

# Configuración de la prueba de doblaje
# NOTA: ¡Los algoritmos O(n^2) se vuelven MUY LENTOS rápido!
# Se redujo de 6400 para que el notebook corra rápido.
sizes = [50, 100, 200, 400, 800, 1600] 

# Casos de prueba
cases = {
    "Average Case": generate_average_case,
    "Best Case": generate_best_case,
    "Worst Case": generate_worst_case
}

results = []


# --- 4. EJECUCIÓN DEL TESTBENCH ---

print("--- Iniciando Testbench de Algoritmos de Ordenamiento ---")

for case_name, generator_func in cases.items():
    print(f"\n--- Probando Caso: {case_name} ---")
    
    for n in sizes:
        print(f"Generando lista de tamaño n={n}...")
        # Genera la lista base para este tamaño y caso
        base_list = generator_func(n)
        
        for algo_name, algo_func in algorithms_to_test:
            
            # ¡IMPORTANTE! Copia la lista para que cada algo ordene la misma
            list_to_sort = base_list.copy() 
            
            print(f"  Ejecutando {algo_name} (n={n})...")
            
            # Ejecuta el profiler
            start_time = time.time()
            sorted_list, comps, moves = algo_func(list_to_sort)
            end_time = time.time()
            
            duration_ms = (end_time - start_time) * 1000
            
            # Almacena el resultado
            results.append({
                "Algorithm": algo_name,
                "Case": case_name,
                "Size (n)": n,
                "Comparisons": comps,
                "Movements": moves,
                "Time (ms)": duration_ms
            })

print("\n--- Testbench completado ---")

# --- 5. CONVERTIR A DATAFRAME ---
# (Esto reemplaza la escritura a CSV)
df = pd.DataFrame(results)

print("\n¡Resultados generados exitosamente en el DataFrame!")
print("Primeras 5 filas:")
display(df.head())

--- Iniciando Testbench de Algoritmos de Ordenamiento ---

--- Probando Caso: Average Case ---
Generando lista de tamaño n=50...
  Ejecutando Bubble Sort (n=50)...
  Ejecutando Selection Sort (n=50)...
  Ejecutando Insertion Sort (n=50)...
Generando lista de tamaño n=100...
  Ejecutando Bubble Sort (n=100)...
  Ejecutando Selection Sort (n=100)...
  Ejecutando Insertion Sort (n=100)...
Generando lista de tamaño n=200...
  Ejecutando Bubble Sort (n=200)...
  Ejecutando Selection Sort (n=200)...
  Ejecutando Insertion Sort (n=200)...
Generando lista de tamaño n=400...
  Ejecutando Bubble Sort (n=400)...
  Ejecutando Selection Sort (n=400)...
  Ejecutando Insertion Sort (n=400)...
Generando lista de tamaño n=800...
  Ejecutando Bubble Sort (n=800)...
  Ejecutando Selection Sort (n=800)...
  Ejecutando Insertion Sort (n=800)...
Generando lista de tamaño n=1600...
  Ejecutando Bubble Sort (n=1600)...
  Ejecutando Selection Sort (n=1600)...
  Ejecutando Insertion Sort (n=1600)...

--- Proban

,Algorithm,Case,Size (n),Comparisons,Movements,Time (ms)
0,Bubble Sort,Average Case,50,1215,741,0.000000
1,Selection Sort,Average Case,50,1225,47,0.000000
2,Insertion Sort,Average Case,50,787,741,0.000000
3,Bubble Sort,Average Case,100,4929,2703,1.000166
4,Selection Sort,Average Case,100,4950,97,0.000000


## 3. Gráficas de Rendimiento (Matplotlib)

A continuación, se generan las gráficas para Tiempo, Comparaciones y Movimientos. 

Cada métrica se divide en 3 sub-gráficas, una para cada caso de prueba (Promedio, Mejor, Peor).

In [4]:
# Función auxiliar para graficar
def plot_metric(df, metric_column, y_label, title):
    # Obtener los casos y algoritmos únicos
    cases = df['Case'].unique()
    algorithms = df['Algorithm'].unique()
    
    # Crear 3 subplots (1 fila, 3 columnas), uno para cada caso
    fig, axes = plt.subplots(1, 3, figsize=(20, 5), sharey=False) # sharey=False para que cada eje Y se ajuste
    fig.suptitle(title, fontsize=16, y=1.02) # Título general
    
    # Mapeo de colores para consistencia entre gráficas
    colors = {
        'Bubble Sort': 'blue',
        'Selection Sort': 'orange',
        'Insertion Sort': 'green'
    }
    
    # Iterar sobre cada caso (Average, Best, Worst) y asignarlo a un subplot
    for i, case in enumerate(cases):
        ax = axes[i]
        df_case = df[df['Case'] == case]
        
        # En cada subplot, pintar una línea por algoritmo
        for algo in algorithms:
            df_algo = df_case[df_case['Algorithm'] == algo]
            ax.plot(
                df_algo['Size (n)'], 
                df_algo[metric_column], 
                label=algo, 
                marker='o', 
                color=colors.get(algo, 'black') # Usar color del mapa
            )
        
        ax.set_title(f"Caso: {case}")
        ax.set_xlabel("Tamaño de la Lista (n)")
        if i == 0: # Solo poner la etiqueta Y en el primer gráfico
            ax.set_ylabel(y_label)
        
        # Aplicar escala logarítmica si la métrica no es 'Time (ms)' y el caso no es 'Best Case'
        # (El mejor caso tiene valores muy bajos o cero, el log da problemas)
        if metric_column != 'Time (ms)' and case != 'Best Case':
             ax.set_yscale('log')
        
        ax.grid(True, which="both", ls="--", alpha=0.5)
        ax.legend()
            
    plt.tight_layout(rect=[0, 0.03, 1, 0.95]) # Ajustar para el supertítulo
    plt.show()

# --- Graficar los resultados ---

# Solo ejecutar si el DataFrame no está vacío
if not df.empty:
    # Gráfica 1: Tiempo
    plot_metric(
        df, 
        'Time (ms)', 
        'Tiempo de Ejecución (ms)', 
        'Tiempo de Ejecución vs. Tamaño de la Lista'
    )
    
    # Gráfica 2: Comparaciones
    plot_metric(
        df, 
        'Comparisons', 
        'Número de Comparaciones (escala log)', 
        'Número de Comparaciones vs. Tamaño de la Lista'
    )
    
    # Gráfica 3: Movimientos
    plot_metric(
        df, 
        'Movements', 
        'Número de Movimientos (escala log)', 
        'Número de Movimientos/Intercambios vs. Tamaño de la Lista'
    )
else:
    print("No se pueden generar gráficos porque no se generaron datos.")

## 4. Conclusiones Clave de los Datos

1.  **Complejidad O(n^2):** En los casos "Promedio" y "Peor", las tres gráficas (Tiempo, Comparaciones y Movimientos) muestran curvas cuadráticas (parábolas). Esto confirma la teoría de que son algoritmos O(n^2). (La escala logarítmica en comparaciones y movimientos ayuda a ver las diferencias, pero la tendencia es claramente no lineal).

2.  **Mejor Caso (¡La gran diferencia!):**
    * **Bubble Sort** e **Insertion Sort** son *adaptativos*. Sus gráficas de Tiempo, Comparaciones y Movimientos son planas (casi cero). Detectan que la lista está ordenada y terminan en tiempo O(n).
    * **Selection Sort** *no es adaptativo*. Su gráfica de Tiempo y Comparaciones en el "Mejor Caso" es idéntica a la del "Caso Promedio" y "Peor Caso". Siempre realiza O(n^2) comparaciones, sin importar el orden inicial.

3.  **Eficiencia en Movimientos:**
    * La gráfica de **Selection Sort** en "Movimientos" es casi plana (lineal, O(n)) en todos los casos. Esto confirma su ventaja única: realiza O(n) intercambios, sin importar qué. Es ideal si las operaciones de "escribir" o "intercambiar" son muy costosas.
    * **Bubble Sort** e **Insertion Sort** realizan O(n^2) movimientos en el peor caso y el promedio.